In [2]:
from ortools.linear_solver import pywraplp

# Define the constants
P = [100, 120, 110]  # Production per turbine in park 1, 2, 3
C_M = [200, 220, 210]  # Cost per turbine in park 1, 2, 3
N = 50  # Maximum number of turbines per park
L = 2  # Number of different cable types
C_L = [
    [1000, 2000],  # Cost for cables 1 and 2 for park 1
    [1100, 2100],  # Cost for cables 1 and 2 for park 2
    [1050, 2050],  # Cost for cables 1 and 2 for park 3
]
Q = [5000, 10000]  # Capacity of cables 1 and 2
D = 10000  # Required amount of delivered electricity
C_I = 10000  # Cost to establish the third wind farm

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables
x = [solver.IntVar(0, N, f'x_{i+1}') for i in range(3)]  # x_1, x_2, x_3
delta = [
    [solver.BoolVar(f'delta_{i+1}_{ell+1}') for ell in range(L)] for i in range(3)
]  # delta_{i}{ell}
y = solver.BoolVar('y')  # Binary variable for the third park

# Constraints

# 1. Total production meets demand
solver.Add(sum(P[i] * x[i] for i in range(3)) >= D)

# 2. Upper limit on the number of turbines per park
for i in range(3):
    solver.Add(x[i] <= N)

# 3. Production in each park must be less than or equal to the capacity of the selected cable
for i in range(3):
    solver.Add(
        P[i] * x[i] <= sum(Q[ell] * delta[i][ell] for ell in range(L))
    )

# 4. Only one cable is used per park
for i in range(3):
    solver.Add(sum(delta[i][ell] for ell in range(L)) == 1)

# 5. If the third park is not built, x_3 = 0
solver.Add(x[2] <= N * y)

# 6. Capacity constraint for parks 1 and 3 combined
solver.Add(
    P[0] * x[0] + P[2] * x[2]
    <= sum(Q[ell] * (delta[0][ell] + delta[2][ell]) for ell in range(L))
)

# Objective function
objective = solver.Objective()

# Cost for turbines
for i in range(3):
    objective.SetCoefficient(x[i], C_M[i])

# Cost for cables
for i in range(3):
    for ell in range(L):
        objective.SetCoefficient(delta[i][ell], C_L[i][ell])

# Cost to establish the third park
objective.SetCoefficient(y, C_I)

objective.SetMinimization()

# Solve the problem
status = solver.Solve()

# Output the solution
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    for i in range(3):
        print(f'Number of turbines in park {i+1}: {x[i].solution_value()}')
    for i in range(3):
        for ell in range(L):
            if delta[i][ell].solution_value() > 0.5:
                print(f'Cable {ell+1} is used for park {i+1}')
    print(f'Is the third park built? {"Yes" if y.solution_value() > 0.5 else "No"}')
    print(f'Minimum total cost: {solver.Objective().Value()}')
else:
    print('The problem does not have an optimal solution.')


Solution:
Number of turbines in park 1: 40.0
Number of turbines in park 2: 50.0
Number of turbines in park 3: 0.0
Cable 1 is used for park 1
Cable 2 is used for park 2
Cable 1 is used for park 3
Is the third park built? No
Minimum total cost: 23150.0
